In [1]:
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, Phi3ForCausalLM

torch.random.manual_seed(0) 
model = Phi3ForCausalLM.from_pretrained( 
    "microsoft/Phi-3-mini-128k-instruct",  
    device_map="mps",  
    torch_dtype="auto",  
    trust_remote_code=True,  
) 


# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct") 

# messages = [ 
#     {"role": "system", "content": "You are a helpful AI assistant."}, 
#     {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"}, 
#     {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."}, 
#     {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"}, 
# ] 

# pipe = pipeline( 
#     "text-generation", 
#     model=model, 
#     tokenizer=tokenizer, 
# ) 

# generation_args = { 
#     "max_new_tokens": 500, 
#     "return_full_text": False, 
#     "temperature": 0.0, 
#     "do_sample": False, 
# } 

# output = pipe(messages, **generation_args) 
# print(output[0]['generated_text']) 



The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out

In [2]:
with torch.no_grad():
    output = model(input_ids=torch.tensor([[1, 2, 3, 4, 5]], device="mps"), output_hidden_states=True)

    sts = output['hidden_states']
    print(len(sts))


# output['hidden_states']


33


In [ ]:
import scipy
import math
def ff(x):
    res = (scipy.special.lambertw(math.exp(-x) * x) + x)
    assert res.imag == 0, "Imaginary part should be 0"
    return res.real
# ff(-.5)


In [11]:
from torch.nn import functional as F

F.silu(torch.tensor(ff(0.12345)))


tensor(0.1234, dtype=torch.float64)

In [3]:
# def solve(A: torch.Tensor, b: torch.Tensor):
#     # solve A x = b for x when it is underdetermined. Return the space of all solutions.
#     # you can assume that A is full rank.
#     # Use SVD to find the null space and particular solution
#     U, S, Vh = torch.linalg.svd(A)
    
#     # Get dimensions
#     m, n = A.shape
#     rank = m  # Since A is full rank
    
#     # Get particular solution using pseudoinverse
#     S_inv = torch.zeros((n, m), device=A.device, dtype=A.dtype)
#     S_inv[:rank, :rank] = torch.diag(1.0 / S[:rank])
#     x_particular = Vh.T @ S_inv @ U.T @ b
    
#     # Get null space basis
#     null_space = Vh[rank:].T
    
#     # Return particular solution and null space basis
#     # Any solution can be written as: x = x_particular + null_space @ c 
#     # where c is any vector of appropriate size
#     return x_particular, null_space


# rand_inp = torch.randn()

Parameter containing:
tensor([[-0.0291, -0.0259,  0.0447,  ...,  0.0099,  0.0425,  0.0603],
        [ 0.0195, -0.0459, -0.0908,  ..., -0.0162,  0.0596, -0.0752],
        [ 0.1040, -0.0381,  0.0332,  ...,  0.0571,  0.1001, -0.0352],
        ...,
        [-0.0146, -0.0015, -0.0120,  ...,  0.0063,  0.0135, -0.0063],
        [-0.0145, -0.0016, -0.0120,  ...,  0.0064,  0.0135, -0.0064],
        [-0.0146, -0.0016, -0.0120,  ...,  0.0063,  0.0134, -0.0064]],
       device='mps:0', dtype=torch.bfloat16, requires_grad=True)

In [2]:
N_DIM = model.lm_head.weight.shape[0]
def sparse_eye(n):
    indices = torch.arange(n).unsqueeze(0).repeat(2, 1)
    values = torch.ones(n)
    return torch.sparse_coo_tensor(indices, values, (n, n))



hyperplanes = torch.cat((-torch.ones((N_DIM-1, 1)).to_sparse(), sparse_eye(N_DIM-1)), dim=-1)

tst = (hyperplanes.to_dense().to('mps') @ model.lm_head.weight)


In [3]:
import numpy as np
import polytope as pc

pp_top = pc.Polytope(hyperplanes.cpu().to_dense().numpy(), np.zeros(N_DIM-1))

pp = pc.Polytope(tst.detach().cpu().numpy(), np.zeros(N_DIM))



In [4]:
len(pp)

0

In [10]:

def reduce(poly, nonEmptyBounded=1, abs_tol=pc.polytope.ABS_TOL):
    """Remove redundant inequalities from the hyperplane representation.

    Uses the algorithm described at [1],
    by solving one LP for each facet.

    [1] https://www.cs.mcgill.ca/~fukuda/soft/polyfaq/node24.html

    Warning:
      - nonEmptyBounded == 0 case is not tested much.

    @type poly: L{Polytope} or L{Region}

    @return: Reduced L{Polytope} or L{Region} object
    """
    if isinstance(poly, pc.Region):
        lst = []
        for poly2 in poly.list_poly:
            red = reduce(poly2)
            if pc.is_fulldim(red):
                lst.append(red)
        if len(lst) > 0:
            return pc.Region(lst, poly.props)
        else:
            return pc.Polytope()
    # is `poly` already in minimal representation ?
    if poly.minrep:
        return poly
    # if not is_fulldim(poly):
    #     return Polytope()
    
    print("aaa")
    # `poly` isn't flat
    A_arr = poly.A
    b_arr = poly.b
    # Remove rows with b = inf
    keep_row = np.nonzero(poly.b != np.inf)
    A_arr = A_arr[keep_row]
    b_arr = b_arr[keep_row]
    neq = np.shape(A_arr)[0]
    # first eliminate the linearly dependent rows
    # corresponding to the same hyperplane
    # Normalize all rows
    a_norm = 1 / np.sqrt(np.sum(A_arr.T**2, 0))
    a_normed = np.dot(A_arr.T, np.diag(a_norm)).T
    remove_row = []
    
    # Compute all pairwise dot products between normalized vectors
    dot_products = np.dot(a_normed, a_normed.T)
    
    # Get indices of parallel hyperplanes (dot product close to 1)
    parallel_pairs = np.where(
        np.triu(dot_products > 1 - abs_tol, k=1)
    )
    
    # For each parallel pair, check which inequality constrains more
    for i, j in zip(*parallel_pairs):
        b_in = b_arr[i] * a_norm[i]
        b_jn = b_arr[j] * a_norm[j]
        if b_in < b_jn:
            remove_row.append(j)
        else:
            remove_row.append(i)


    keep_row = np.setdiff1d(range(neq), remove_row).tolist()
    A_arr = A_arr[keep_row]
    b_arr = b_arr[keep_row]
    neq, nx = A_arr.shape
    print(f"new shape: {A_arr.shape}")
    print("bbb")
    if nonEmptyBounded:
        if neq <= nx + 1:
            return pc.Polytope(A_arr, b_arr)
    # # Now eliminate hyperplanes outside the bounding box
    # if neq > 3 * nx:
    #     lb, ub = pc.Polytope(A_arr, b_arr).bounding_box
    #     # Do a coordinate system translation such that the lower bound is
    #     # moved to the origin
    #     #       A*(x-lb) <= b - A*lb
    #     # Relative to the origin, a row ai in A with only positive coefficients
    #     # represents an upper bound. If ai*(x1-lb) <= bi,
    #     # the hyperplane is above x1.
    #     # Hence, if ai*(ub-lb) <= bi, then the hyperplane at row i
    #     # does not intersect the bounding box.
    #     # The same holds for rows with negative coefficients multiplied with
    #     # the origin. Rows with both negative and positive coefficients
    #     # are a mixture of the two extremes.
    #     cand = ~ (np.dot((A_arr > 0) * A_arr, ub - lb) -
    #               (np.array([b_arr]).T - np.dot(A_arr, lb)) < -1e-4)
    #     A_arr = A_arr[cand.squeeze()]
    #     b_arr = b_arr[cand.squeeze()]
    print("ccc")
    neq, nx = A_arr.shape
    if nonEmptyBounded:
        if neq <= nx + 1:
            return pc.Polytope(A_arr, b_arr)
    # Check for each inequality whether it is implied by
    # the other inequalities, i.e., is it redundant?
    del keep_row[:]
    for k in range(neq):
        print("ddd ", k)
        # Setup object function to maximize the linear function
        # defined as current row of A matrix
        f = -A_arr[k, :]
        G = A_arr
        h = b_arr
        # Give some slack in the current inequality
        h[k] += 0.1
        sol = pc.polytope.lpsolve(f, G, h)
        h[k] -= 0.1
        if sol['status'] == 0:
            # If the maximum is greater than the constraint of
            # the inequality, then the inequality constrains solutions
            # and thus the inequality is non-redundant
            obj = -sol['fun'] - h[k]
            if obj > abs_tol:
                keep_row.append(k)
        elif sol['status'] == 3:
            keep_row.append(k)
    polyOut = pc.Polytope(A_arr[keep_row], b_arr[keep_row])
    polyOut.minrep = True
    return polyOut

In [11]:
test_res = reduce(pp)

aaa


In [ ]:
# pp.contains()

In [12]:
pp.A

array([[ 0.014  , -0.00577, -0.03903, ..., -0.00753,  0.00492, -0.03903],
       [ 0.042  , -0.00385, -0.00362, ...,  0.01491,  0.01819, -0.03013],
       [-0.02164,  0.00614,  0.01621, ..., -0.0196 , -0.03234, -0.02635],
       ...,
       [ 0.00614,  0.01033, -0.02404, ..., -0.0015 , -0.01228, -0.02829],
       [ 0.00617,  0.01031, -0.02404, ..., -0.00146, -0.01231, -0.0283 ],
       [ 0.00614,  0.0103 , -0.02404, ..., -0.00152, -0.01233, -0.0283 ]])

In [5]:
one_top = torch.cat((torch.ones(1), torch.zeros(N_DIM-1))).cpu().type(torch.DoubleTensor)

with torch.no_grad():
    sln = torch.linalg.lstsq(model.lm_head.weight.cpu().type(torch.DoubleTensor), one_top)

In [13]:
pp_top.contains(one_top[:, None])

array([ True])

In [15]:
pp.contains(sln.solution[:, None])

array([ True])

In [42]:
tst.cpu() @ sln.solution.type(torch.float)

tensor([-0.0720, -0.0677, -0.0846,  ..., -0.0868, -0.0868, -0.0868],
       grad_fn=<MvBackward0>)

In [28]:
test_res = model.lm_head(sln.solution.type(torch.bfloat16).to('mps'))

tensor(0, device='mps:0')

In [44]:
hyperplanes @ test_res.cpu().type(torch.float32), \
    torch.all(hyperplanes @ test_res.cpu().type(torch.float32) < 0)

(tensor([-0.0718, -0.0675, -0.0844,  ..., -0.0866, -0.0866, -0.0866],
        grad_fn=<MvBackward0>),
 tensor(True))